## Решение основной задачи:

Для начала необходимо импортировать необходимые библиотеки:

In [ ]:
import numpy as np
from tensorflow.keras.datasets import imdb
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, BatchNormalization
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau
from tensorflow.keras.regularizers import l2
from sklearn.utils.class_weight import compute_class_weight

Далее необходимо загрузить набор данных IMDB, указав `num_words=10000`, который указывает, что нужно оставить только 10 000 самых частых слов в отзывах.

In [ ]:
max_words = 10000 #устанавливаем размер словаря

(x_train, y_train), (x_test, y_test) = imdb.load_data(num_words=max_words) #загрузка данных

17464789/17464789 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step


Далее выполним бинарную векторизацию текстовых данных (в данном случае — отзывов IMDB) с помощью метода **one-hot encoding**:

In [ ]:
word_index = imdb.get_word_index()
reverse_word_index = dict([(value, key) for (key, value) in word_index.items()])
decoded_review = ' '.join([reverse_word_index.get(i-3, '?') for i in x_train[0]])

1641221/1641221 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step


Напишем функцию, которая выполняет векторизацию данных - преобразование в бинарные векторы:

In [ ]:
def vectorize_sequences(sequences, dimension=10000):
    results = np.zeros((len(sequences), dimension)) #cоздаём нулевую матрицу размером (число отзывов × dimension)
    for i, sequence in enumerate(sequences):
        results[i, sequence] = 1.
    return results

Подготавливаем данные для обучения. Применяем функцию **vectorize_sequences**, преобразовываем метки в тип данных **float32**:

In [ ]:
x_train = vectorize_sequences(x_train)
x_test = vectorize_sequences(x_test)
y_train = np.asarray(y_train).astype('float32')
y_test = np.asarray(y_test).astype('float32')

Нормализуем входные данные, поделив их на самое максимальное значение их набора данных. Это делается для для масштабирования входных признаков в диапазон [0, 1].

In [ ]:
#нормализация входных данных
x_train = x_train / x_train.max()
x_test = x_test / x_test.max()

Далее вычислим веса классов (**class weights**) для борьбы с дисбалансом классов в данных:

In [ ]:
#вычисление весов классов
class_weights = compute_class_weight('balanced', classes=np.unique(y_train), y=y_train)
class_weights = dict(enumerate(class_weights))

Затем создадим последовательную (**Sequential**) модель нейронной сети для бинарной классификации текстов. Функциями активации будут `relu` и `sigmoid`. Также будем использовать `BatchNormalization()`. Этот метод , который позволяет повысить производительность и стабилизировать работу модели. К первым трём слоям Dense применим l2-регуляризацию с коэффициентом 0.002. Также применим прореживание. Прореживание, которое применяется к слою, заключается в удалении (присваивании нуля) случайно выбираемым признакам на этапе обучения.

In [ ]:
model = Sequential([
    Dense(256, activation='relu', kernel_regularizer=l2(0.002), input_shape=(max_words,)),
    BatchNormalization(),
    Dropout(0.7),

    Dense(128, activation='relu', kernel_regularizer=l2(0.002)),
    BatchNormalization(),
    Dropout(0.6),

    Dense(64, activation='relu', kernel_regularizer=l2(0.002)),
    BatchNormalization(),
    Dropout(0.5),

    Dense(32, activation='relu'),
    BatchNormalization(),

    Dense(1, activation='sigmoid')
])

/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Создадим оптимизатор **Adam**, установив параметр `learning_rate=0.0002`, отвечающий за шаги обучения. Так же начнем компилировать модель, установив параметр функции потерь для бинарной классификации, оптимизатор и метрику:

In [ ]:
optimizer = Adam(learning_rate=0.0002) #создание оптимизатора
model.compile(optimizer=optimizer, loss='binary_crossentropy', metrics=['accuracy']) #компиляция модели

**EarlyStopping** - ранняя остановка обучения. Это нужно, чтобы автоматически останавливать обучение, когда модель перестаёт улучшаться, предотвращая переобучение и экономя время. Мониторит метрику `val_accuracy` (точность на валидационных данных). Если точность не улучшается в течение `patience=10` эпох — обучение останавливается. `restore_best_weights=True` — после остановки модель возвращает веса, которые давали наилучшую `val_accuracy`, а не последние веса.

**ReduceLROnPlateau** - уменьшение **learning rate**. Мониторит `val_loss` (функцию потерь на валидации). Если ошибка не уменьшается в течение patience=5 эпох:
*   Текущий learning rate умножается на factor=0.3;
*   Если улучшений нет после нескольких уменьшений, learning rate не упадёт ниже min_lr=1e-6.

In [ ]:
early_stopping = EarlyStopping(monitor='val_accuracy', patience=10, restore_best_weights=True) #callback-функция EarlyStopping
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.3, patience=5, min_lr=1e-6) #callback-функция ReduceLROnPlateau

Далее переходим к обучении модели. Установим 80 эпох, количество обучающих примеров, используемых за одну итерацию при обучении нейронной сети = 128, 20% данных уйдёт на валидационную выборку:

In [ ]:
history = model.fit(
    x_train, y_train,
    epochs=80,
    batch_size=128,
    validation_split=0.2,
    class_weight=class_weights,
    callbacks=[early_stopping, reduce_lr],
    verbose=1
)

Epoch 1/80
157/157 ━━━━━━━━━━━━━━━━━━━━ 12s 79ms/step - accuracy: 0.9237 - loss: 0.9056 - val_accuracy: 0.8904 - val_loss: 0.9444 - learning_rate: 2.0000e-04
Epoch 2/80
157/157 ━━━━━━━━━━━━━━━━━━━━ 20s 78ms/step - accuracy: 0.9334 - loss: 0.8123 - val_accuracy: 0.8872 - val_loss: 0.8963 - learning_rate: 2.0000e-04
Epoch 3/80
157/157 ━━━━━━━━━━━━━━━━━━━━ 20s 72ms/step - accuracy: 0.9380 - loss: 0.7418 - val_accuracy: 0.8852 - val_loss: 0.8567 - learning_rate: 2.0000e-04
Epoch 4/80
157/157 ━━━━━━━━━━━━━━━━━━━━ 20s 71ms/step - accuracy: 0.9458 - loss: 0.6791 - val_accuracy: 0.8870 - val_loss: 0.8283 - learning_rate: 2.0000e-04
Epoch 5/80
157/157 ━━━━━━━━━━━━━━━━━━━━ 12s 74ms/step - accuracy: 0.9541 - loss: 0.6177 - val_accuracy: 0.8840 - val_loss: 0.8105 - learning_rate: 2.0000e-04
Epoch 6/80
157/157 ━━━━━━━━━━━━━━━━━━━━ 20s 75ms/step - accuracy: 0.9543 - loss: 0.5753 - val_accuracy: 0.8846 - val_loss: 0.7894 - learning_rate: 2.0000e-04
Epoch 7/80
157/157 ━━━━━━━━━━━━━━━━━━━━ 20s 70ms/ste

Выведем результат обучения. Для выполнения задачи важно отобразить валидационную и контрольную выборки. [1] используется для получения значения точности (**accuracy**) из результатов метода `model.evaluate()`:

In [ ]:
val_accuracy = model.evaluate(x_train, y_train)[1]
test_accuracy = model.evaluate(x_test, y_test)[1]

print(f"Валидационная выборка - {val_accuracy:.4f}")
print(f"Контрольная выборка - {test_accuracy:.4f}")

782/782 ━━━━━━━━━━━━━━━━━━━━ 8s 10ms/step - accuracy: 0.9683 - loss: 0.7500
782/782 ━━━━━━━━━━━━━━━━━━━━ 9s 11ms/step - accuracy: 0.8807 - loss: 0.9612
Валидационная выборка - 0.9537
Контрольная выборка - 0.8821
